In [1]:
import os
from tqdm import tqdm
import pandas as pd
import mido
from mido import MidiFile, MidiTrack, Message, MetaMessage
import pretty_midi
import numpy as np

In [2]:
MDBS_MIDI = {
    "KD": 35,
    "SD": 38,
    "SDB": 38,
    "SDD": 38,
    "SDF": 38,
    "SDG": 38,
    "SDNS": 38,
    "CHH": 42,
    "OHH": 46,
    "PHH": 44,
    "HIT": 50,
    "MHT": 48,
    "HFT": 43,
    "LFT": 41,
    "RDC": 51,
    "RDB": 53,
    "CRC": 49,
    "CHC": 52,
    "SPC": 55,
    "SST": 37,
    "TMB": 54, 
}

In [3]:
mix_folder = 'D:/Github/phd-drum-sep/data/MDBDrums-master/MDB Drums/audio/full_mix/'
mixes = os.listdir(mix_folder)
mixes = [mix_folder + m for m in mixes]

In [4]:
drum_folder = 'D:/Github/phd-drum-sep/data/MDBDrums-master/MDB Drums/audio/drum_only/'
drum = os.listdir(drum_folder)
drum = [drum_folder + d for d in drum]

In [5]:
beats_folder = 'D:/Github/phd-drum-sep/data/MDBDrums-master/MDB Drums/annotations/beats/'
beats = os.listdir(beats_folder)
beats = [beats_folder + b for b in beats]

In [6]:
class_folder = 'D:/Github/phd-drum-sep/data/MDBDrums-master/MDB Drums/annotations/subclass/'
classes = os.listdir(class_folder)
classes = [class_folder + c for c in classes]

In [7]:
for idx, val in tqdm(enumerate(classes)):

    name = val.replace('D:/Github/phd-drum-sep/data/MDBDrums-master/MDB Drums/annotations/subclass/', '')
    name = name.replace('_subclass.txt', '')

    f = open(beats[idx], "r")
    b = f.read()

    f = open(classes[idx], "r")
    c = f.read()
    

    bets = b.replace('\n', '	').split('	')
    bs = []
    for idx, val in enumerate(bets):
        if idx % 2 == 0:
            bs.append(val)
    
    bs = bs[:len(bs) - 1]
    bs = [float(b) for b in bs]

    clsses = c.replace("\n", "\t").replace(" ", "")
    clsses = clsses.split("\t")
    times = []
    cl = []
    for idx, val in enumerate(clsses):
        if idx % 2 == 0:
            times.append(val)
        else:
            cl.append(val)
    
    if len(times) != len(cl):
        times = times[:len(times) - 1]
    
    times = [float(t) for t in times]
    
    clsses = cl
    times = times
    bets = bs
    
    # Create a new MIDI file and a MIDI track for drums
    # Create a PrettyMIDI object
    midi_object = pretty_midi.PrettyMIDI()
    
    # Create an Instrument instance for a drum set
    # In General MIDI, program number 0 with is_acoustic_drum=True represents the standard drum set
    drum_program = pretty_midi.instrument_name_to_program('Acoustic Grand Piano')
    drum_instrument = pretty_midi.Instrument(program=drum_program, is_drum=True)
    
    # Add notes to the drum instrument
    for cls, time in zip(clsses, times):
        note_number = MDBS_MIDI[cls]
        # Create a Note instance for each drum hit
        note = pretty_midi.Note(
            velocity=100,  # Adjust velocity as needed
            pitch=note_number,
            start=time,
            end=time + 0.1  # Adjust note duration as needed
        )
        drum_instrument.notes.append(note)
    
    # Add the drum instrument to the PrettyMIDI object
    midi_object.instruments.append(drum_instrument)

    
    # Save the MIDI file
    midi_file_name = f'D:/Github/phd-drum-sep/data/MDBDrums-master/MDB Drums/midi/{name}.mid'
    midi_object.write(midi_file_name)


23it [00:00, 96.24it/s]


In [8]:
for note in midi_object.instruments[0].notes:
    print(note)

Note(start=0.000000, end=0.100000, pitch=49, velocity=100)
Note(start=0.004354, end=0.104354, pitch=35, velocity=100)
Note(start=0.290000, end=0.390000, pitch=42, velocity=100)
Note(start=0.420000, end=0.520000, pitch=35, velocity=100)
Note(start=0.554376, end=0.654376, pitch=42, velocity=100)
Note(start=0.560000, end=0.660000, pitch=38, velocity=100)
Note(start=0.840000, end=0.940000, pitch=42, velocity=100)
Note(start=1.110000, end=1.210000, pitch=42, velocity=100)
Note(start=1.260000, end=1.360000, pitch=35, velocity=100)
Note(start=1.400000, end=1.500000, pitch=42, velocity=100)
Note(start=1.530000, end=1.630000, pitch=35, velocity=100)
Note(start=1.670000, end=1.770000, pitch=42, velocity=100)
Note(start=1.800000, end=1.900000, pitch=35, velocity=100)
Note(start=1.920726, end=2.020726, pitch=42, velocity=100)
Note(start=1.930000, end=2.030000, pitch=38, velocity=100)
Note(start=2.192834, end=2.292834, pitch=42, velocity=100)
Note(start=2.200000, end=2.300000, pitch=35, velocity=10